In [1]:
import sys
sys.path.append('../')

from utils import save_data_with_pickle, load_data_with_pickle
from CorpusManager import CorpusManager
from tqdm.notebook import tqdm

In [2]:
CORPUS_PATH = '/datahdd/vmanuel/ELMo/Corpora/shuffled_text_with_words'

c = CorpusManager()
c.read_corpus(CORPUS_PATH, 100000)

read input corpus


100%|██████████| 100000/100000 [00:04<00:00, 20694.01it/s]


In [3]:
PICKLE_PATH = ('../../../source_files/pickles/')

found_entity_dict = load_data_with_pickle(PICKLE_PATH + 'found_entity_dict_9_3')


In [ ]:
found_entity_dict

In [ ]:
c.joined_corpus[51634]

In [ ]:
word_indexes = load_data_with_pickle(PICKLE_PATH + 'word_occurrence_index_32150_100000static')

In [ ]:
word_indexes.keys()

In [ ]:
from collections import defaultdict
reverse_dict = defaultdict(list)

entity_dict = load_data_with_pickle(PICKLE_PATH + 'entity_dict_9_3')

for k, words in entity_dict.items():
    for w in words:
        reverse_dict[w].append(k)

In [ ]:
entity_dict

In [ ]:
all_entities = []

for k, v in entity_dict.items():
    all_entities.extend(v)
all_entities = set(all_entities)

In [ ]:
from colorama import Fore, Back, Style

In [ ]:
def check_entity_in_row(ENTITY, ROW, verbose):
    
    a = extract_rows_occurrency(ENTITY.split(' '), ROW)
    if verbose:
        print('row: {}'.format(ROW))
        print('entity: {}'.format(ENTITY))
        print('indexes of each word: {}'.format([(w, o) for w, o in zip(ENTITY.split(' '), a)]))
        print(' '.join([Fore.BLACK + c_w if c_w not in ENTITY.split(' ') else Fore.RED + c_w for c_w in c.corpus[ROW]]))
        print(Fore.BLACK + '--------')
    values = []
    for value in a[0]:
        occ = [(value, 0)]
        i = 1
        while i < len(a) and value + i in a[i]:
            occ.append(((value + i), i))
            i += 1
        if i == len(a):
            if verbose:
                print(Fore.BLACK + 'entity present in this sentence at the index {}'.format(value))
            values.append(value)
        else:
            if verbose:
                print(Fore.BLACK + 'entity not present in this sentence at the index {}'.format(value))
        if verbose:
            print(Fore.BLACK + '--------')
    if values:
        return values

In [ ]:
from tqdm.notebook import tqdm
keys = set(word_indexes.keys())

def extract_rows(word):
    return [t[0] for t in word_indexes[word]] #extract the row index from each tuple

def extract_rows_occurrency(word_phrase, row):
    return [t[1] for s in word_phrase for t in word_indexes[s] if t[0] == row]

j = 0
verbose = False
found = defaultdict(list)
for entity in tqdm(all_entities):
    splitted = entity.split(' ')
    if set(splitted) <= keys:
        # all words in this entity are in the corpus
        i = 0
        rows = extract_rows(splitted[i])
        while rows and i + 1 < len(splitted):
            i += 1
            rows = [r for r in rows if r in extract_rows(splitted[i])]
        if rows and len(splitted) > 1:
            for r in rows:
                b = check_entity_in_row(ENTITY=entity, ROW=r, verbose = verbose)
            if b:
                if verbose:
                    print('entity found at index(es): {}'.format(' '.join([str(v) for v in b])))
                found[entity].append((r, b))
            j +=1
        elif rows:
            found[entity] = word_indexes[entity]
print(found)

In [ ]:
all_occurrences = load_data_with_pickle(PICKLE_PATH + 'occurrences_of_entities')
all_occurrences = [(k , v) if type(v[0]) == tuple else (k, v[0]) for k, v in all_occurrences.items() if len(k) > 2]
all_occurrences = {x[0]: x[1] for x in all_occurrences}

In [ ]:
all_occurrences

In [ ]:
sentences_to_embed = [v[0] for values in all_occurrences.values() for v in values]

print('total found entity mentions: {}'.format(len(sentences_to_embed)))
print('fraction of sentences with entity mentions: {:.2f} ({} on {})'.format(len(set(sentences_to_embed))/len(c.joined_corpus),
                                                                             len(set(sentences_to_embed)),
                                                                             len(c.joined_corpus)))
print('{:.2f} average entity mentions per sentence'.format(len(sentences_to_embed)/len(set(sentences_to_embed))))

In [ ]:
all_occurrences['the']

In [ ]:
embedding_data_structure = {index:[] for index in sentences_to_embed if index < 10000}

for entity_mention, occurrences in all_occurrences.items():
    for couple in occurrences:
        if couple[0] < 10000:
            embedding_data_structure[couple[0]].append((couple[1], entity_mention))
                    
from collections import OrderedDict


ordered_embedding_data_structure = OrderedDict(sorted(embedding_data_structure.items()))
ordered_embedding_data_structure

In [ ]:
from allennlp.commands.elmo import ElmoEmbedder

elmo = ElmoEmbedder(cuda_device = 0)

In [ ]:
ordered_embedding_data_structure

In [ ]:
import numpy as np
vectors_dict = defaultdict(list)

for row_index, occurrences in tqdm(ordered_embedding_data_structure.items()):
    vectors = elmo.embed_sentence(c.corpus[row_index])[2]
    for occ in occurrences:
        for word_index in occ[0]:
            if len(occ[1].split(' ')) == 1:
                vectors_dict[occ[1]].append(vectors[word_index])
            else:
                vecs = [vectors[w_i] for w_i in range(word_index, word_index + len(occ[1].split(' ')))]
                vectors_dict[occ[1]].append(np.mean(vecs, axis = 0))                        
# print('{}'.format('\n'.join(['{}'.format((k, len(v))) for k, v in vectors_dict.items()])))

In [ ]:
found_entities = set(vectors_dict.keys())
found_entity_dict = {k: set(v).intersection(found_entities) for k,v in entity_dict.items() if set(v).intersection(found_entities)}    

In [ ]:
found_entity_dict

In [ ]:
G = load_data_with_pickle(PICKLE_PATH + 'graph')

found_entity_dict_2 = c.avoid_multilabeling(found_entity_dict, G, file = 'avoid_multilabeling.txt')

In [ ]:
all_occurrences.keys()

In [ ]:
found_entity_dict

In [ ]:
reverse_dict = defaultdict(list)

# entity_dict = load_data_with_pickle(PICKLE_PATH + 'entity_dict')

for k, words in found_entity_dict.items():
    for w in words:
        reverse_dict[w].append(k)

In [ ]:
len(reverse_dict)

In [ ]:
reverse_dict['the']

In [ ]:
X = []
Y = []
entities = []

for label, label_vectors in vectors_dict.items():
    if label in reverse_dict:
        for v in label_vectors:
            X.append(v)
            Y.append(reverse_dict[label])
            entities.append(label)

In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
len(entities)

In [ ]:
save_data_with_pickle(PICKLE_PATH + '../vectors/elmo_vectors', X)
save_data_with_pickle(PICKLE_PATH + '../vectors/labels', Y)
save_data_with_pickle(PICKLE_PATH + '../vectors/entities', entities)